In [ ]:
pip install underthesea

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.9/20.9 MB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 657.8/657.8 kB 10.4 MB/s eta 0:00:00


In [ ]:
import json
import os
import re
from underthesea import word_tokenize

# Mục mới

In [ ]:
def load_all_data(data_path):
    all_data = []
    for filename in os.listdir(data_path):
        if filename.endswith('.json'):
            file_path = os.path.join(data_path, filename)
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
                all_data.extend(data['data'])
    return {'data': all_data}

In [ ]:

def clean_text(text):
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\[\d+\]', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = text.lower()
    return text

In [ ]:
def preprocess_data(data):
    contexts = []
    questions = []
    answers = []
    for article in data['data']:
        for paragraph in article['paragraphs']:
            context = clean_text(paragraph['context'])
            for qa in paragraph['qas']:
                question = clean_text(qa['question'])
                is_impossible = qa.get('is_impossible', False)  #Do ngữ liệu nhiều chỗ define thiếu is_impossible nên sẽ mặc định là False
                if not is_impossible:
                    for answer in qa['answers']:
                        answer_text = clean_text(answer['text'])
                        answer_start = answer['answer_start']
                        contexts.append(context)
                        questions.append(question)
                        answers.append({
                            'text': answer_text,
                            'start': answer_start
                        })
    return contexts, questions, answers

In [ ]:
def tokenize_texts(texts):
 return [word_tokenize(text, format="text") for text in texts]

In [ ]:
data_path = '/content/data'
squad_data = load_all_data(data_path)
contexts, questions, answers = preprocess_data(squad_data)
tokenized_contexts = tokenize_texts(contexts)
tokenized_questions = tokenize_texts(questions)

In [ ]:
tokenized_questions[:100]

In [ ]:
pip install gensim

In [ ]:
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess


In [ ]:
# Tokenize ngữ liệu
split_context_pre_train = [context.split() for context in tokenized_contexts]
split_question_pre_train = [question.split() for question in tokenized_questions]

In [ ]:
split_question_pre_train[100:110]

In [ ]:
# Tạo mô hình Word2Vec
model_word2vec = Word2Vec(vector_size=100, window=10, min_count=1, sg=1, workers=4)

In [ ]:
# Xây dựng từ điển
model_word2vec.build_vocab(split_question_pre_train)

In [ ]:
model_word2vec.train(split_question_pre_train, total_examples=model_word2vec.corpus_count, epochs=100)

(1952236, 3667600)

In [ ]:
# Lưu mô hình Word2Vec đã huấn luyện
model_word2vec.save("/content/word2vec.model")

In [ ]:
similarWord = model_word2vec.wv.most_similar('hợp_đồng', topn=5)
print(similarWord)

[('chấm_dứt', 0.5507318377494812), ('muốn', 0.5413239002227783), ('trở_thành', 0.5283077955245972), ('mới', 0.5205602049827576), ('báo', 0.5093007683753967)]


In [ ]:
!pip install rank_bm25

In [ ]:
from rank_bm25 import BM25Okapi

# Tạo BM25 model
bm25 = BM25Okapi(split_question_pre_train)

In [ ]:
questionBm25 = "hợp đồng lao động"
tokenized_query = tokenize_texts([questionBm25])
print(tokenized_query)

['hợp_đồng lao_động']


In [ ]:
# Tính điểm số BM25

bm25_scores = bm25.get_scores(tokenized_query)
print(bm25_scores)

[0. 0. 0. ... 0. 0. 0.]


In [ ]:
import numpy as np

# Xếp hạng các câu hỏi dựa trên điểm số
top_n = np.argsort(bm25_scores)[::-1][:10]

# Hiển thị các câu hỏi có điểm số cao nhất
print("Câu hỏi người dùng:", questionBm25)
print("Các câu hỏi có điểm số cao nhất:")
for index in top_n:
    print(f"Điểm số: {bm25_scores[index]:.4f} - Câu hỏi: {split_question_pre_train[index]}")


Câu hỏi người dùng: bảo hiểm y tế
Các câu hỏi có điểm số cao nhất:
Điểm số: 0.0000 - Câu hỏi: ['người', 'sử_dụng', 'lao_động', 'có', 'nghĩa_vụ', 'gì', 'nếu', 'tranh_chấp', 'lao_động', 'phát_sinh', 'sau', 'thời_gian', 'thử', 'việc']
Điểm số: 0.0000 - Câu hỏi: ['người', 'sử_dụng', 'lao_động', 'phải', 'giải_quyết', 'các', 'tranh_chấp', 'phát_sinh', 'trong', 'quá_trình', 'thực_hiện', 'thỏa_ước', 'lao_động', 'tập_thể', 'theo', 'điều', 'gì']
Điểm số: 0.0000 - Câu hỏi: ['tổ_chức', 'đại_diện', 'người', 'lao_động', 'có', 'trách_nhiệm', 'gì', 'trong', 'việc', 'giải_quyết', 'tranh_chấp', 'lao_động', 'tập_thể']
Điểm số: 0.0000 - Câu hỏi: ['người', 'sử_dụng', 'lao_động', 'phải', 'tuân_thủ', 'các', 'quy_định', 'của', 'pháp_luật', 'về', 'điều', 'gì']
Điểm số: 0.0000 - Câu hỏi: ['người', 'sử_dụng', 'lao_động', 'có', 'nghĩa_vụ', 'gì', 'trong', 'việc', 'giải_quyết', 'tranh_chấp', 'lao_động', 'tập_thể']
Điểm số: 0.0000 - Câu hỏi: ['người', 'lao_động', 'có_thể', 'yêu_cầu', 'ai', 'giải_quyết', 'tranh_chấp'

In [284]:
# Changed
def get_avg_word2vec_vector(user_question):
    words = user_question
    word_vectors = [model_word2vec.wv[word] for word in words if word in model_word2vec.wv]
    if not word_vectors:
        return np.zeros(model_word2vec.vector_size)
    return np.mean(word_vectors, axis=0)

In [295]:
# Changed

def get_word2vec_scores(user_question, questions):
    query_vector = get_avg_word2vec_vector(user_question)
    scores = []
    for question in questions:
        question_vector = get_avg_word2vec_vector(simple_preprocess(question))
        score = np.dot(query_vector, question_vector)
        if np.isnan(score):
            score = 0
        scores.append(score)
    return np.array(scores)

In [288]:
# Added
def get_bm25_scores(user_question):
    bm_25_score = bm25.get_scores(user_question)
    scores = (bm_25_score - np.min(bm_25_score)) / (np.max(bm_25_score) - np.min(bm_25_score))
    # Thay thế giá trị nan bằng 0
    scores = np.nan_to_num(scores)
    return scores

In [289]:
# Added
def get_combined_scores(user_question, questions):
    tokenized_query = simple_preprocess(tokenize_texts([user_question])[0])
    word2vec_scores = get_word2vec_scores(tokenized_query, questions)
    bm25_scores = get_bm25_scores(tokenized_query)
    word2vec_scores = (word2vec_scores - np.min(word2vec_scores)) / (np.max(word2vec_scores) - np.min(word2vec_scores))
    combined_scores = word2vec_scores + bm25_scores
    # Thay thế giá trị nan bằng 0 trong combined_scores
    combined_scores = np.nan_to_num(combined_scores)
    return combined_scores

In [290]:
def find_best_matching_question(user_question, questions):
    combined_scores = get_combined_scores(user_question, questions)
    if np.all(combined_scores == 0):
        return "Không có câu hỏi được tìm thấy"
    best_match_idx = np.argmax(combined_scores)
    return questions[best_match_idx]

In [296]:
input_question = "Người lao động tham gia mấy loại bảo hiểm nào"
best_question = find_best_matching_question(input_question, questions)

print(f"User Question: {input_question}")
print(f"Best matching question: {best_question}")

User Question: Người lao động tham gia mấy loại bảo hiểm nào
Best matching question: người sử dụng lao động phải tham gia loại bảo hiểm nào cho lao động đặc thù theo quy định của pháp luật
